# 创建Plugins

In [16]:
from semantic_kernel.functions import kernel_function
from typing import Annotated

class DestinationPlugin:
    """A List of Destinations for vacation."""
    
    @kernel_function(description="Provides a list of of vacation destionations.")
    def get_destinations(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Paris, France
        Tokyo, Japan
        New York City, USA
        Sydney, Australia
        Cape Town, South Africa
        Rome, Italy
        Barcelona, Spain
        Bali, Indonesia
        Dubai, UAE
        Santorini, Greece
        """
    
    @kernel_function(description="Provides the availability of a destination.")
    def get_availability(self, destination: Annotated[str, "The destination to check availability for."]) -> Annotated[str, "Returns the availability of the destination."]:
        return """
        Paris - Unavailable
        Tokyo - Available
        New York City - Available
        Sydney - Unavailable
        Cape Town - Available
        Rome - Unavailable
        Barcelona - Unavailable
        Bali - Available
        Dubai - Unavailable
        Santorini - Available
        """

In [17]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

load_dotenv()

client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com/"
)

kernel = Kernel()
kernel.add_plugin(plugin=DestinationPlugin(), plugin_name="destinations")

service_id = "travel_agent"

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    service_id=service_id,
    async_client=client
)

kernel.add_service(chat_completion_service)

# 设置Funciton Choice Behavior

In [18]:
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# 创建Agent

In [19]:
from semantic_kernel.functions import KernelArguments
from semantic_kernel.agents import ChatCompletionAgent

AGENT_NAME  = "TravelAgent"
AGENT_INSTRCTIONs = "Answer questions about the travel destinations and their availability."

trave_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRCTIONs,
    arguments=KernelArguments(settings=settings)
)

# 运行Agent

In [20]:
from IPython.display import display, HTML

from semantic_kernel.contents import ChatHistory, FunctionCallContent, FunctionResultContent

async def main():
    chat_history = ChatHistory()

    user_inputs = [
        "What destinations are available?",
        "Is Barcelona available?",
        "Are there any vacation destinations available not in Europe?",
    ]

    for user_input in user_inputs:
        chat_history.add_user_message(user_input)

        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""
        function_calls = []

        async for content in trave_agent.invoke_stream(history=chat_history):
            if not agent_name and hasattr(content, "name"):
                agent_name = content.name
            
            for item in content.items:
                if isinstance(item, FunctionCallContent):
                    call_info = f"Calling: {item.function_name}({item.arguments})"
                    function_calls.append(call_info)
                elif isinstance(item, FunctionResultContent):
                    result_info = f"Result: {item.result}"
                    function_calls.append(result_info)
            
            if(hasattr(content, "content") and 
               content.content and 
               content.content.strip() and 
               not any(isinstance(item, (FunctionCallContent, FunctionResultContent)) for item in content.items)):
                full_response += content.content
            
            # Add function calls to HTML if any occurred
        if function_calls:
            html_output += f"<div style='margin-bottom:10px'>"
            html_output += f"<details>"
            html_output += f"<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            html_output += f"<div style='margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
            html_output += "<br>".join(function_calls)
            html_output += f"</div></details></div>"

        # Add agent response to HTML
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()